# [Chapter 3. Numbers, Dates, and Times](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html)

Performing mathematical calculations with integers and floating-point numbers is easy in Python.  
However, if you need to perform calculations with fractions, arrays, or dates and times, a bit more work is required.  
The focus of this chapter is on such topics.

## [Rounding Numerical Values](http://chimera.labs.oreilly.com/books/1230000000393/ch03.html#_rounding_numerical_values)